In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_MATH_CONFIG, SMOL_LM_135M

cfg = GRPO_MATH_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
# cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

In [9]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-03-22 19:40:31.694 | INFO     | trl_wrapper.trainer_wrapper:__init__:249 - Setting padding side to: left
2025-03-22 19:40:31.702 | INFO     | trl_wrapper.trainer_wrapper:init_model:260 - Using device: cuda:0
2025-03-22 19:40:31.703 | INFO     | trl_wrapper.trainer_wrapper:init_model:274 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: flash_attention_2


In [10]:
wrapper.init_data_module()

2025-03-22 19:40:33.453 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:300 - Using chat template override: smollmv2
2025-03-22 19:40:33.454 | INFO     | trl_wrapper.wrapper_config:__init__:131 - Cache dir: ../dataset_caches/g_s_m8_k_data_module


In [11]:
wrapper.init_trainer()

2025-03-22 19:40:35.782 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:337 - Saving output to: ./runs/03-22-19-40-882269-smollm2-135m-instruct-
2025-03-22 19:40:35.785 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:360 - Initializing trainer, run_name: 03-22-19-40-882269-smollm2-135m-instruct-, wandb project: qwen-math-grpo
2025-03-22 19:40:35.786 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:445 - Using vllm: False


In [12]:
from model.reasoning import SYSTEM_PROMPT

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {
        "role": "user",
        "content": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_attention_mask=True
)
tokenized_chat = tokenized_chat.to("cuda")
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

system

You are an expert reasoning model.
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

# Example

User: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?
Assistant: <reasoning>
Tony got twice $1750 which is 2*$1750 = 3500
The total amount shared was $1750+$3500 = 5250
</reasoning>
<answer>5250</answer>

# Example

User: Bob has a collection of marbles. If his friend Tim has 20 marbles and Bob has three times as many marbles as Tim, how many marbles do they have together?
Assistant: <reasoning>Bob has 3x20 = 60 marbles.
Together, they have 20 + 60 = 80 marbles.</reasoning>
<answer>80</answer>


user
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
assistant
In April, Natalia sold 48 clips, and in May, she sold 1/2 of that number.
I

In [13]:
wrapper.train()

2025-03-22 19:40:44.160 | INFO     | trl_wrapper.trainer_wrapper:train:614 - Starting training.
2025-03-22 19:40:48.190 | INFO     | model.reasoning:xmlcount_reward_func:91 - XML count rewards: [0.0, 0.0]
2025-03-22 19:40:48.191 | INFO     | model.reasoning:soft_format_reward_func:69 - Soft format rewards: [0.0, 0.0]
2025-03-22 19:40:48.192 | INFO     | model.reasoning:strict_format_reward_func:59 - Strict format rewards: [0.0, 0.0]
2025-03-22 19:40:48.193 | INFO     | model.reasoning:int_reward_func:49 - Integer rewards: [0.0, 0.0]
2025-03-22 19:40:48.194 | INFO     | model.reasoning:correctness_reward_func:22 - --------------------
Question:
Sheena is sewing the bridesmaid’s dresses for her sister’s wedding. She can sew one dress in 12 hours. There are 5 bridesmaids in the wedding. If Sheena sews the dresses 4 hours each week, how many weeks will it take her to complete them?
Answer:
15
Generated:
Sheena can sew one dress in 12 hours, so in 12 hours, Sheena can sew 12/12 = 1 dress.
I

[{'content': '\nYou are an expert reasoning model.\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n\n# Example\n\nUser: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\nAssistant: <reasoning>\nTony got twice $1750 which is 2*$1750 = 3500\nThe total amount shared was $1750+$3500 = 5250\n</reasoning>\n<answer>5250</answer>\n\n# Example\n\nUser: Bob has a collection of marbles. If his friend Tim has 20 marbles and Bob has three times as many marbles as Tim, how many marbles do they have together?\nAssistant: <reasoning>Bob has 3x20 = 60 marbles.\nTogether, they have 20 + 60 = 80 marbles.</reasoning>\n<answer>80</answer>\n\n', 'role': 'system'}, {'content': 'Sheena is sewing the bridesmaid’s dresses for her sister’s wedding. She can sew one dress in 12 hours. There are 5 bridesmaids in the wedding. If Sheena sews the dresses 4 hours e

2025-03-22 19:40:52.594 | INFO     | model.reasoning:xmlcount_reward_func:91 - XML count rewards: [0.0, 0.0]
2025-03-22 19:40:52.596 | INFO     | model.reasoning:soft_format_reward_func:69 - Soft format rewards: [0.0, 0.0]
2025-03-22 19:40:52.598 | INFO     | model.reasoning:strict_format_reward_func:59 - Strict format rewards: [0.0, 0.0]
2025-03-22 19:40:52.600 | INFO     | model.reasoning:int_reward_func:49 - Integer rewards: [0.0, 0.0]
2025-03-22 19:40:52.603 | INFO     | model.reasoning:correctness_reward_func:22 - --------------------
Question:
James buys 3 shirts for $60.  There is a 40% off sale.  How much did he pay per shirt after the discount?
Answer:
12
Generated:
Let's break down the problem:

James has 3 shirts for $60. 
The discount is 40% off. 
The discount is 40% of 3 shirts, which is 0.4 * 3 = 1.2.
So, after the discount, James will pay $60 - 1.2 * 3 = $60 - 4.8 = $42.80.

So, James paid $42.80 for the shirts.
Extracted:
Let's break down the problem:

James has 3 shirt

[{'content': '\nYou are an expert reasoning model.\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n\n# Example\n\nUser: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\nAssistant: <reasoning>\nTony got twice $1750 which is 2*$1750 = 3500\nThe total amount shared was $1750+$3500 = 5250\n</reasoning>\n<answer>5250</answer>\n\n# Example\n\nUser: Bob has a collection of marbles. If his friend Tim has 20 marbles and Bob has three times as many marbles as Tim, how many marbles do they have together?\nAssistant: <reasoning>Bob has 3x20 = 60 marbles.\nTogether, they have 20 + 60 = 80 marbles.</reasoning>\n<answer>80</answer>\n\n', 'role': 'system'}, {'content': 'James buys 3 shirts for $60.  There is a 40% off sale.  How much did he pay per shirt after the discount?', 'role': 'user'}]


2025-03-22 19:40:57.204 | INFO     | model.reasoning:xmlcount_reward_func:91 - XML count rewards: [0.125, 0.0]
2025-03-22 19:40:57.206 | INFO     | model.reasoning:soft_format_reward_func:69 - Soft format rewards: [0.0, 0.0]
2025-03-22 19:40:57.207 | INFO     | model.reasoning:strict_format_reward_func:59 - Strict format rewards: [0.0, 0.0]
2025-03-22 19:40:57.208 | INFO     | model.reasoning:int_reward_func:49 - Integer rewards: [0.5, 0.0]
2025-03-22 19:40:57.209 | INFO     | model.reasoning:correctness_reward_func:22 - --------------------
Question:
A phone factory makes twice as many phones as last year. Last year’s production was 5000 phones. If a quarter of this year’s production is sold, how many phones are left in the factory?
Answer:
7500
Generated:
Last year's production was 5000 phones.
Last year's production was 20% of this year's production, so 5000 / (20% of 5000) = 5000 / (20*100) = 5000 / 200 = 2500
2500 phones are left in the factory.
</reasoning>
<answer>2500</answer>


[{'content': '\nYou are an expert reasoning model.\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n\n# Example\n\nUser: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\nAssistant: <reasoning>\nTony got twice $1750 which is 2*$1750 = 3500\nThe total amount shared was $1750+$3500 = 5250\n</reasoning>\n<answer>5250</answer>\n\n# Example\n\nUser: Bob has a collection of marbles. If his friend Tim has 20 marbles and Bob has three times as many marbles as Tim, how many marbles do they have together?\nAssistant: <reasoning>Bob has 3x20 = 60 marbles.\nTogether, they have 20 + 60 = 80 marbles.</reasoning>\n<answer>80</answer>\n\n', 'role': 'system'}, {'content': 'A phone factory makes twice as many phones as last year. Last year’s production was 5000 phones. If a quarter of this year’s production is sold, how many phones are left in the fa

KeyboardInterrupt: 